In [43]:
import pandas as pd
import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error

In [11]:
#TODO: Get the values for the features from the API
transactions = pd.read_csv("Dummy Data/Transactions_2022Q1.csv")
transactions = transactions.sort_values(by="postDate")
transactions.head()

,type,id,status,description,amount,account,balance,direction,class,institution,...,enrich,transactionDate,postDate,subClass,links.self,links.account,links.institution,links.connection,subClass.title,subClass.code
349,transaction,11a6f99b-3448-41ec-8514-436ed7f7d78e,posted,WARMUN ROADHOUSE EAST ARM AU,-114.40,49544383-c19c-4c97-b8d1-5322d0faa6af,-38.10,debit,payment,AU00000,...,NaN,NaN,2022-01-01T00:00:00Z,NaN,https://au-api.basiq.io/users/11103cba-4a08-43...,https://au-api.basiq.io/users/11103cba-4a08-43...,https://au-api.basiq.io/institutions/AU00000,NaN,Supermarket and Grocery Stores,411.0
348,transaction,5a6171f6-6d33-4fb7-87ec-05e58dabc853,posted,COLES EXPRESS 6952 HALLS CREEK AU,-326.01,49544383-c19c-4c97-b8d1-5322d0faa6af,-364.11,debit,payment,AU00000,...,NaN,NaN,2022-01-01T00:00:00Z,NaN,https://au-api.basiq.io/users/11103cba-4a08-43...,https://au-api.basiq.io/users/11103cba-4a08-43...,https://au-api.basiq.io/institutions/AU00000,NaN,Fuel Retailing,400.0
347,transaction,d0d696f2-21b7-4b41-9972-af5b557383e4,posted,TARUNDA SUPERMARKET FITZROY CROS AU,-10.04,49544383-c19c-4c97-b8d1-5322d0faa6af,-374.15,debit,payment,AU00000,...,NaN,NaN,2022-01-01T00:00:00Z,NaN,https://au-api.basiq.io/users/11103cba-4a08-43...,https://au-api.basiq.io/users/11103cba-4a08-43...,https://au-api.basiq.io/institutions/AU00000,NaN,Supermarket and Grocery Stores,411.0
346,transaction,0b713af7-6aa0-4945-ae8f-4218b27f14eb,posted,CABLE BEACH CARAVAN CABLE BEACH AU,-131.00,49544383-c19c-4c97-b8d1-5322d0faa6af,-505.15,debit,payment,AU00000,...,NaN,NaN,2022-01-02T00:00:00Z,NaN,https://au-api.basiq.io/users/11103cba-4a08-43...,https://au-api.basiq.io/users/11103cba-4a08-43...,https://au-api.basiq.io/institutions/AU00000,NaN,Accommodation,440.0
345,transaction,6f8f5796-7121-49c2-8977-ba17b862595c,posted,CAFE DAMORE PTY LTD BROOME AU,-70.00,49544383-c19c-4c97-b8d1-5322d0faa6af,-575.15,debit,payment,AU00000,...,NaN,NaN,2022-01-02T00:00:00Z,NaN,https://au-api.basiq.io/users/11103cba-4a08-43...,https://au-api.basiq.io/users/11103cba-4a08-43...,https://au-api.basiq.io/institutions/AU00000,NaN,"Cafes, Restaurants and Takeaway Food Services",451.0


In [25]:
testPercent = 0.1 #10% training
predictorColumn = "balance"

split_idx = int((1-testPercent) * len(transactions))

train_data = transactions.iloc[:split_idx]
test_data = transactions.iloc[split_idx:]

X_train = train_data.drop(predictorColumn, axis=1)
X_test = test_data.drop(predictorColumn, axis=1)
y_train = train_data[[predictorColumn]]
y_test = test_data[[predictorColumn]]

In [52]:
#Add new models here, you can seperately find hyperparameters to best optimse them
models = [
    SARIMAX(y_train, order = (1, 0, 1)),
    SARIMAX(y_train, order = (0, 1, 0))
]

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


In [53]:
fits = []

for m in models:
    fits.append(m.fit())

In [55]:
predictions = []

for f in fits:
    predictions.append(f.predict(start=len(y_train), end=len(y_train)+len(y_test)-1))

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


In [56]:
meanSquareErrors = []

for p in predictions:
    meanSquareErrors.append(mean_squared_error(y_test, p))
    
meanSquareErrors

[3464513815.425554, 3476113595.5576305]

In [58]:
totalMSE = sum(meanSquareErrors)
weights = []

for mse in meanSquareErrors:
    weights.append((totalMSE - mse)/totalMSE)
    
weights

[0.5008356434833053, 0.4991643565166947]